# 19 

In [559]:
patterns = []
designs = []

with open("input.txt", "r", encoding="utf-8") as file: 
    for id, line in enumerate(file):
        if id == 0: 
            patterns = [pattern.strip() for pattern in line.strip().split(",")]
        else: 
            if line.strip() == "": 
                continue

            designs.append(line.strip())

In [513]:
print(patterns)

['rrgbg', 'rgguubg', 'rbru', 'rb', 'rrrw', 'wbu', 'gbgb', 'uururg', 'ubru', 'rugb', 'bbru', 'b', 'rggurg', 'wgru', 'bgrwb', 'rrgubg', 'ubrrbg', 'wgurru', 'rrrrw', 'rbrwu', 'wubwb', 'wrbbr', 'bgbu', 'brrww', 'brg', 'gbwu', 'wrubuur', 'gur', 'grbr', 'gruwrrbg', 'bgwr', 'wgugbgb', 'rwbru', 'wuwr', 'rrg', 'ruwg', 'rgwgr', 'ubu', 'wbr', 'bwg', 'gbu', 'bgrwrb', 'wwrw', 'bb', 'gr', 'rug', 'grr', 'ubwb', 'rbruwbu', 'guggug', 'ugu', 'rwbuu', 'bbur', 'wbrww', 'wubw', 'br', 'gruu', 'gwr', 'wrrwwu', 'wug', 'bgu', 'bgb', 'wugb', 'grb', 'rbbg', 'grgbwgb', 'rwu', 'bwrbb', 'uwrbbru', 'bbu', 'wrg', 'rwg', 'ggbgbg', 'wbwru', 'wrgrw', 'bburr', 'bgr', 'rgbg', 'bwwwugrb', 'uwu', 'rwuwrg', 'gguburu', 'uwubwu', 'wb', 'wgguug', 'ugrbw', 'rbgu', 'rwwu', 'ggbbgb', 'gw', 'wgg', 'rrw', 'uurg', 'uubg', 'bgurgb', 'uwgbr', 'wbrwu', 'rrbb', 'rww', 'bwgb', 'uugru', 'ggrbub', 'wubgw', 'rwwugr', 'uuwur', 'burbrg', 'bbr', 'ggb', 'wgr', 'gwbgw', 'rrbu', 'wuwwrwb', 'rbbr', 'uubw', 'buuu', 'ubbwggu', 'brr', 'urguww', 'uggww

First, I implemented a silly solution (below) but then got another idea: look for an inteval "cover" of the string, then turn that into a graph and check if there is a path from start to finish.

This can be done "dynamically" while building the cover. 

In [514]:
import re 

def design_cover(design, patterns):
    """
    Gets all intervals that are covered by the patterns in the design. 
    """
    intervals = []
    for pattern in patterns: 
        regex = re.compile(pattern)
        for pattern_match in re.finditer(regex, design):
            intervals.append((pattern_match.start(), pattern_match.end()))
    return intervals

In [515]:
def is_covered(design, patterns):
    """ 
    Checks if the design is covered by the patterns.
    """

    # Get a list of intervals over the design 
    edges = design_cover(design, patterns)

    # Create a graph from the intervals
    adjacency = dict()  # out edges

    for edge in edges: 
        if edge[0] not in adjacency:
            adjacency[edge[0]] = [edge[1]]
        else:
            adjacency[edge[0]].append(edge[1])
            
    # All of this is fast! 

    # DFS 
    stack = [0]
    discovered = set()

    while len(stack) > 0:
        current = stack.pop()

        if current == len(design): 
            return True

        if current in discovered: 
            continue
        else:
            discovered.add(current)

            if current in adjacency:
                stack.extend(adjacency[current])

    return False


result = 0

for design in designs: 
    result += is_covered(design, patterns)

print(result)

371


In [516]:
# Another solution, a bit more straightforward.
def find_path(patterns, design): 
    if len(design) == 0: 
        return True
    for pattern in patterns: 
        if design.startswith(pattern):
            if find_path(patterns, design[len(pattern):]):
                return True
    return False

result = 0
for design in designs:
    if find_path(patterns, design):
        result += 1

result

371

Dynamic programming? 

# Part 2

Count the number of all paths by traversing from end to start? 

I thought that this could be done by removing the discovery check in DFS, but ufff.... 

TODO: finish the path counting... the adjacency mtx somehow doesn't work and I suspect I have a mistake in the way I build the regex somehow :(

In [497]:
import numpy as np

In [500]:
def num_designs(designs, patterns):
    res = 0
    for design in designs:
        edges = design_cover(design, patterns)

        #print(edges)

        # Adjacency matrix 
        matrix = np.zeros((len(design)+1, len(design)+1))
        for node, out in edges:
            matrix[node, out] += 1

        init_vector = np.zeros((len(design)+1))
        init_vector[0] = 1

        m = matrix.copy().T
        for _ in range(len(design)+10):
            res += init_vector[-1]
            init_vector = m @ init_vector
        
    return int(res)

Wrong answer (too low): 430126150705147. I still wonder why is this wrong though :( 

In [446]:
430126150705147 == 430126150705147.0

True

In [562]:
memo = {}

def count_paths(patterns, design, memo): 
    if design in memo: return memo[design] 
    if len(design) == 0: return memo.get(design, 1) # This was the issue! 
    for pattern in patterns: 
        if design.startswith(pattern):
            memo[design] = memo.get(design,0) + count_paths(patterns, design[len(pattern):], memo)
    return memo.get(design, 0)

result = 0
for design in designs:
    result += count_paths(patterns, design, memo)

result

650354687260341

In [563]:
from functools import cache